In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
! pip install scipy==1.1.0
%tensorflow_version 1.x

     |████████████████████████████████| 31.2MB 141kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
TensorFlow 1.x selected.


In [0]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import tensorflow as tf
from scipy.misc import imread, imsave
from skimage.segmentation import clear_border
from skimage.morphology import label
from skimage.measure import regionprops
from skimage.transform import resize

In [0]:
class Extract_Letters:
    def extractFile(self, filename):
        image = imread(filename, 1)
    
        #apply threshold in order to make the image binary
        bw = image < 120
    
        # remove artifacts connected to image border
        cleared = bw.copy()
        #clear_border(cleared)

        # label image regions
        label_image = label(cleared,neighbors=8)
        borders = np.logical_xor(bw, cleared)
        label_image[borders] = -1
    
    
        #fig = plt.figure(figsize=(6,6))
        #ax = fig.add_subplot(131)
        #ax.imshow(bw, cmap='jet')

        letters = list()
        order = list()
    
        for region in regionprops(label_image):
            minc, minr, maxc, maxr = region.bbox
            # skip small images
            if maxc - minc > len(image)/250: # better to use height rather than area.
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='red', linewidth=2)
                order.append(region.bbox)


        #sort the detected characters left->right, top->bottom
        lines = list()
        first_in_line = ''
        counter = 0

        #worst case scenario there can be 1 character per line
        for x in range(len(order)):
            lines.append([])
    
        for character in order:
            if first_in_line == '':
                first_in_line = character
                lines[counter].append(character)
            elif abs(character[0] - first_in_line[0]) < (first_in_line[2] - first_in_line[0]):
                lines[counter].append(character)
            elif abs(character[0] - first_in_line[0]) > (first_in_line[2] - first_in_line[0]):
                first_in_line = character
                counter += 1
                lines[counter].append(character)


        for x in range(len(lines)):       
            lines[x].sort(key=lambda tup: tup[1])

        final = list()
        prev_tr = 0
        prev_line_br = 0
        
        for i in range(len(lines)):
            for j in range(len(lines[i])):
                tl_2 = lines[i][j][1]
                bl_2 = lines[i][j][0]
                if tl_2 > prev_tr and bl_2 > prev_line_br:
                    tl,tr,bl,br = lines[i][j]
                    letter_raw = bw[tl:bl,tr:br]
                    letter_norm = resize(letter_raw ,(20 ,20))
                    final.append(letter_norm)
                    prev_tr = lines[i][j][3]
                if j == (len(lines[i])-1):
                    prev_line_br = lines[i][j][2]
            prev_tr = 0
            tl_2 = 0

        print ("Characters recognized: " + str(len(final)))
        return final


    def __init__(self):
        print ("Extracting characters...")

In [0]:
# to make this notebook's output stable across runs
def reset_graph(seed=123):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [0]:
n_inputs = 20*20 #size of box around letter
n_hidden1 = 52
n_outputs = 26 #number of different characters
learning_rate = 0.05

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y") 

In [8]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [9]:
start_time = time.time()
extract = Extract_Letters()
training_files = ['./gdrive/My Drive/COM2028_Coursework/Training_Data/A.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/B.png', 
                  './gdrive/My Drive/COM2028_Coursework/Training_Data/C.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/D.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/E.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/F.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/G.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/H.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/I.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/J.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/K.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/L.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/M.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/N.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/O.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/P.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/Q.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/R.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/S.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/T.png',
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/U.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/V.png',
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/W.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/X.png', 
									'./gdrive/My Drive/COM2028_Coursework/Training_Data/Y.png', './gdrive/My Drive/COM2028_Coursework/Training_Data/Z.png']

alphabet = 'abcdefghijklmnopqrstuvwxyz'
count = 0

X_train = None
X_valid = None
y_train = None
y_valid = None

for files in training_files:
	letters = extract.extractFile(files)
 
	values = np.array(letters)
	values = values.astype(np.float32).reshape(-1, 20*20)
	valid_values, train_values = values[:20], values[20:]
	labels = np.full(len(letters), count)
	valid_labels, train_labels = labels[:20], labels[20:]

	if count == 0:
		X_train = train_values
		y_train = train_labels
		X_valid = valid_values
		y_valid = valid_labels
	else:
		X_train = np.concatenate((X_train, train_values))
		y_train = np.concatenate((y_train, train_labels))
		X_valid = np.concatenate((X_valid, valid_values))
		y_valid = np.concatenate((y_valid, valid_labels))

	count += 1
 
print(time.time() - start_time, "seconds" )


Extracting characters...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=8, use connectivity=2
  del sys.path[0]


Characters recognized: 643
Characters recognized: 668
Characters recognized: 672
Characters recognized: 671
Characters recognized: 696
Characters recognized: 821
Characters recognized: 650
Characters recognized: 667
Characters recognized: 1029
Characters recognized: 954
Characters recognized: 670
Characters recognized: 879
Characters recognized: 540
Characters recognized: 622
Characters recognized: 670
Characters recognized: 701
Characters recognized: 677
Characters recognized: 724
Characters recognized: 726
Characters recognized: 798
Characters recognized: 660
Characters recognized: 672
Characters recognized: 517
Characters recognized: 639
Characters recognized: 669
Characters recognized: 719
27.683340549468994 seconds


In [0]:
n_epochs = 20
batch_size = 500

In [43]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.35952848 Validation accuracy: 0.4
1 Batch accuracy: 0.6208252 Validation accuracy: 0.74423075
2 Batch accuracy: 0.69155204 Validation accuracy: 0.8384615
3 Batch accuracy: 0.697446 Validation accuracy: 0.875
4 Batch accuracy: 0.73477405 Validation accuracy: 0.89807695
5 Batch accuracy: 0.7721022 Validation accuracy: 0.9307692
6 Batch accuracy: 0.7426326 Validation accuracy: 0.95
7 Batch accuracy: 0.75442046 Validation accuracy: 0.9653846
8 Batch accuracy: 0.7426326 Validation accuracy: 0.97884613
9 Batch accuracy: 0.78781927 Validation accuracy: 0.98846155
10 Batch accuracy: 0.7976424 Validation accuracy: 0.99038464
11 Batch accuracy: 0.78389 Validation accuracy: 0.98653847
12 Batch accuracy: 0.8015717 Validation accuracy: 0.99423075
13 Batch accuracy: 0.80943024 Validation accuracy: 0.99423075
14 Batch accuracy: 0.81925344 Validation accuracy: 0.99423075
15 Batch accuracy: 0.8310413 Validation accuracy: 0.99423075
16 Batch accuracy: 0.84675837 Validation accuracy: 

In [44]:
letters = extract.extractFile('./gdrive/My Drive/COM2028_Coursework/Testing_Data/test_normal.png')
 
X_test = np.array(letters)
X_test = X_test.astype(np.float32).reshape(-1, 20*20)


Characters recognized: 72


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=8, use connectivity=2
  del sys.path[0]


In [50]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_new_scaled = X_test
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
    print("Predicted letters: [", end = "" )
    for pred in y_pred:
      print(alphabet[pred] + " ", end = "")
    print("]")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
Predicted letters: [t h g g r g a t g s t g i o r y i a i i g i n g i i g s n o t i n n g g g r f a i i i n g j g u t i n g l s o g c g v g g y t o g e w g f a i i ]
